---

# 🧭 JSON Schema Generation

### 🎯 Intent

Generate **standards-compliant JSON Schema** from Pydantic v2 models for **API docs, validation, and contracts** (e.g., FastAPI, OpenAPI).

---

### 🧩 Core Components

1. **📜 `.model_json_schema()`**

   * Produces a JSON Schema dict for a model.
   * Options:

     * `by_alias=True` → use field aliases
     * `ref_template` → customize `$ref` paths
     * `mode="validation"` / `"serialization"` → match input vs output rules
   * Includes `$defs` for shared/nested models.

2. **🧰 `TypeAdapter(...).json_schema()`**

   * Generates schema for plain types (`list[int]`, `Annotated[...]`).
   * Useful when endpoints accept raw arrays or tuples.

3. **🏷 Field Metadata → Schema**

   * Titles, descriptions, examples, and `deprecated=True` flow into schema.
   * Constraints (`min_length`, `max_length`, `ge`, `le`, `pattern`) map automatically.

4. **🧱 Enums, Literals, Unions**

   * `Enum` / `Literal[...]` → `enum` arrays.
   * Unions → `oneOf` / `anyOf`.
   * Discriminated unions → `oneOf` + discriminator.

5. **🔗 Aliases & Naming**

   * Aliases (`by_alias=True`) control property keys.
   * Ensures stable contracts across versions.

6. **🧩 Nested Models & Reuse**

   * Nested models referenced via `$ref` into `$defs`.
   * Avoids duplication, keeps schemas DRY.

7. **⚙️ Model Config for Schema**

   * `json_schema_extra` → inject or override schema details.
   * Extend with custom keys (`x-order`, `x-format`).

8. **🧪 Serialization-Aware Schema**

   * Use `mode="serialization"` for response schemas.
   * Ensures datetimes, UUIDs, Decimals are shown as strings.

9. **🌐 FastAPI / OpenAPI Fit**

   * FastAPI consumes `model_json_schema()` under the hood.
   * Adding titles, descriptions, examples improves docs automatically.

---
